nfringement/concern: Riders Cap 118 and Cap 166 rising together
Time period: Since 01:11:44 AM (EEST) 08/02/19
Location: Longega
GPS Co-Ordinates (DD system): 46.714433, 11.889570
Tracking information from Trackleaders:

In [ ]:
import requests
import pandas as pd 
import folium
from folium import FeatureGroup, features, CircleMarker
from matplotlib import colors as mcolors
import matplotlib
import matplotlib.pyplot as mp
from haversine import haversine
import numpy as np
from dateutil import relativedelta
from math import radians, sin, cos, asin, atan2, degrees

In [ ]:
def calculate_bearing(point1, point2):
        lat1 = point1[0]
        lon1 = point1[1]

        lat2 = point2[0]
        lon2 = point2[1]
        lat1, lon1, lat2, lon2 = map(radians, (lat1, lon1, lat2, lon2))
        y = sin(lon2-lon1)*cos(lat2);
        x = cos(lat1)*sin(lat2) - sin(lat1)*cos(lat2)*cos(lon2-lon1);
        bearing = degrees(atan2(y, x))
        return bearing

In [ ]:
def calculate_coordinates(departure_point, bearing, distance):
        R = earth_radius = 6371
        d = distance
        lat1 = radians(departure_point[0]) #Current lat point converted to radians
        lon1 = radians(departure_point[1]) #Current long point converted to radians
        bearing = radians(bearing)
        lat2 = asin( sin(lat1)*cos(d/R) + cos(lat1)*sin(d/R)*cos(bearing))
        lon2 = lon1 + atan2(sin(bearing)*sin(d/R)*cos(lat1), cos(d/R)-sin(lat1)*sin(lat2))
        lat2 = degrees(lat2)
        lon2 = degrees(lon2)
        return (round(lat2, 4), round(lon2, 4))

In [ ]:
mypath='./2019-08-10/'

In [ ]:
racers = pd.read_pickle('racers.pkl')

In [ ]:
racers[racers.Bib.isin(['118','166'])]

In [ ]:
names = list(racers[racers.Bib.isin(['118','166'])].Name.values)
names

In [ ]:
route={}
for racer in names:
    route[racer] = [[i,l,t] for i,(l,t) in enumerate(pd.read_pickle(mypath+racer+'.pkl').values.tolist())]

In [ ]:
def popup(name,t):
    folium.Popup(name+'\n'+str(t))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
[CircleMarker(l,popup=popup('166',t),radius=1).add_to(m) for i,l,t in route['Lorenza_Walker']]
[CircleMarker(l,color='red',popup=popup('118',t),radius=1).add_to(m) for i,l,t in route['Ghirtoi_Daniel']]
m.fit_bounds(m.get_bounds())
m

In [ ]:
v0 =[(haversine(l,l1)/(t1-t).seconds*3600) for (i,l,t),(i1,l1,t1) in zip(route['Lorenza_Walker'][:-2],route['Lorenza_Walker'][1:])]
v1 =[(haversine(l,l1)/(t1-t).seconds*3600) for (i,l,t),(i1,l1,t1) in zip(route['Ghirtoi_Daniel'][:-2],route['Ghirtoi_Daniel'][1:])]

In [ ]:
dates0 = matplotlib.dates.date2num([t for (i,l,t) in route['Lorenza_Walker'][2:]])
dates1 = matplotlib.dates.date2num([t for (i,l,t) in route['Ghirtoi_Daniel'][2:]])

In [ ]:
fig= mp.figure(figsize=(12,3))
axes= fig.add_axes([0.1,0.1,0.8,0.8])
axes.plot_date(dates0,v0,markersize=2)
axes.plot_date(dates1,v1,markersize=2)
mp.show()

In [ ]:

t0 = [t for (j,l,t) in route['Lorenza_Walker']]
l0 = [l for (j,l,t) in route['Lorenza_Walker']]

t1 = [t1 for (j,l1,t1) in route['Ghirtoi_Daniel']]
l1 = [l1 for (j,l1,t1) in route['Ghirtoi_Daniel']]

In [ ]:
ii =[]
for i,t in enumerate(t0[:-10]):
    j=np.argmin([abs(t-t1) for t1 in t1])
    if (t-t1[j]).seconds > 0:
        d = haversine(l0[i],l0[i+1])/(t0[i+1]-t).seconds*(t-t1[j]).seconds # approx speed and estimate distance
    else:
        if i==0:
            continue
        d = haversine(l0[i],l0[i-1])(t-t[i-1]).seconds*(t-t1[j]).seconds # approx speed and estimate distance
        b =calculate_bearing(l0[i],l1[j])
    print(haversine(l0[i],l1[j]), t, t1[j],(t-t1[j]).seconds,d, haversine(calculate_coordinates(l0[i],b,d),l1[j]))
#    ii.append(np.argmin([abs(t-t1) for (j,l1,t1) in route['Ghirtoi_Daniel']]))                

In [ ]:
t0 = [t for (j,l,t) in route['Lorenza_Walker']]
l0 = [l for (j,l,t) in route['Lorenza_Walker']]

t1 = [t1 for (j,l1,t1) in route['Lorenza_Walker']]
l1 = [l1 for (j,l1,t1) in route['Lorenza_Walker']]

In [ ]:
for i,t in enumerate(t0[:-10]):
    j=np.argmin([abs(t-t1) for t1 in t1])
    if (t-t1[j]).seconds > 0:
        d = haversine(l0[i],l0[i+1])/(t0[i+1]-t).seconds*(t-t1[j]).seconds # approx speed and estimate distance
    else:
        if i==0:
            continue
        d = haversine(l0[i],l0[i-1])/(t-t0[i-1]).seconds*(t-t1[j]).seconds # approx speed and estimate distance
        b =calculate_bearing(l0[i],l1[j])
    print(haversine(l0[i],l1[j]), t, t1[j],(t-t1[j]).seconds,d, haversine(calculate_coordinates(l0[i],b,d),l1[j]))

In [ ]:
route['Lorenza_Walker'][-2:]

In [ ]:
route['Ghirtoi_Daniel'][-2:]

In [ ]:
[point1,point2] =[r[1] for r in  route['Ghirtoi_Daniel'][-2:]]

In [ ]:
calculate_bearing(point1, point2)

In [ ]:
[t1,t2] =[r[2] for r in  route['Ghirtoi_Daniel'][-2:]]

In [ ]:
bearing = calculate_bearing(point1,point2)
bearing

In [ ]:
np = calculate_coordinates(point1,bearing,float((t2-t1).seconds)/3600)
np

In [ ]:
haversine(point2,np)

In [ ]:
[t1a,t2a] =[r[2] for r in  route['Lorenza_Walker'][-2:]]

In [ ]:
calculate_coordinates(point1,bearing,haversine(point1,point2))

In [ ]:
haversine(point1,np)

In [ ]:
haversine(point1,point2)